In [271]:
# coding: utf-8

# Recommendation systems
#
# Here we'll implement a content-based recommendation algorithm.
# It will use the list of genres for a music as the content.
# The data comes from the Lastfm project and Spotify Music

# Please only use these imports.
from collections import Counter, defaultdict
import math

import numpy as np
import os
import pandas as pd
import re
from scipy.sparse import csr_matrix

In [274]:
def tokenize_string(my_string):
    """ 
    This is used in tokenize function.
    """
    return re.findall('[\w\&]+', my_string.lower())

In [275]:
def tokenize(songs):
    """
    Append a new column to the songs DataFrame with header 'tokens'.
    This will contain a list of strings, one per token, extracted
    from the 'genre' field of each song. Use the tokenize_string method above.
    Note: you may modify the songs parameter directly; no need to make
    a new copy.
    Params:
      songs...The songs DataFrame
    Returns:
      The songs DataFrame, augmented to include a new column called 'tokens'.
    >>> songs = pd.DataFrame([[linkin park, 'Alternative'], [Baby, 'Rock']], columns=['songId', 'genres'])
    >>> songs = tokenize(songs)
    >>> songs['tokens'].tolist()
    [['Alternative', 'rock']]
    """
    tokenlist=[]
    for index,row in songs.iterrows():
        tokenlist.append(row.title)
    songs['tokens']=tokenlist
    return songs

In [290]:
def featurize(songs):
    """
    Append a new column to the songs DataFrame with header 'features'.
    Each row will contain a csr_matrix of shape (1, num_features). Each
    entry in this matrix will contain the tf-idf value of the term, as
    defined in class:
    tfidf(i, d) := tf(i, d) / max_k tf(k, d) * log10(N/df(i))
    where:
    i is a term
    d is a document (song)
    tf(i, d) is the frequency of term i in document d
    max_k tf(k, d) is the maximum frequency of any term in document d
    N is the number of documents (songs)
    df(i) is the number of unique documents containing term i
    Params:
      songs...The songs DataFrame
    Returns:
      A tuple containing:
      - The songs DataFrame, which has been modified to include a column named 'features'.
      - The vocab, a dict from term to int. Make sure the vocab is sorted alphabetically as in a2 (e.g., {'aardvark': 0, 'boy': 1, ...})
    """
    def tf(word,doc):
        return doc.count(word) / Counter(doc).most_common()[0][1]

    def df(word, doclist):
        return sum(1 for d in doclist if word in d)

    def tfidf(word, doc, dfdict, N):
        return tf(word, doc) * math.log10((N/dfdict[word]))

    def getcsrmatrix(tokens,dfdict,N,vocab):
        matrixRow_list = []
        matrixRow_list = np.zeros((1,len(vocab)),dtype='float')
        for t in tokens:
            if t in vocab:
                matrixRow_list[0][vocab[t]] = tfidf(t,tokens,dfdict,N)
        return csr_matrix(matrixRow_list)

    N=len(songs)
    doclist = songs['tokens'].tolist()
    vocab = { s:x for x,s in enumerate(sorted(list(set(s for s in doclist)))) }
    dfdict = {}
    for v in vocab.items():
        dfdict[v[0]] = df(v[0],doclist)

    csrlist = []
    for index, row in songs.iterrows():
         csrlist.append(getcsrmatrix(row['tokens'],dfdict,N,vocab))

    songs['features'] =  csrlist
    return (songs,vocab)

In [291]:
def train_test_split(ratings):
    """
    Returns a random split of the ratings matrix into a training and testing set.
    """
    test = set(range(len(ratings))[::1000])
    train = sorted(set(range(len(ratings))) - test)
    test = sorted(test)
    return ratings.iloc[train], ratings.iloc[test]

In [292]:
def cosine_sim(a, b):
    """
    Compute the cosine similarity between two 1-d csr_matrices.
    Each matrix represents the tf-idf feature vector of a movie.
    Params:
      a...A csr_matrix with shape (1, number_features)
      b...A csr_matrix with shape (1, number_features)
    Returns:
      The cosine similarity, defined as: dot(a, b) / ||a|| * ||b||
      where ||a|| indicates the Euclidean norm (aka L2 norm) of vector a.
    """
    v1 = a.toarray()[0]
    v2  = b.toarray()[0]
    return sum(i[0] * i[1] for i in zip(v1, v2))/(math.sqrt(sum([i*i for i in v1]))*math.sqrt(sum([i*i for i in v2])))

In [293]:
def make_predictions(songs, ratings_train, ratings_test):
    """
    Using the ratings in ratings_train, predict the ratings for each
    row in ratings_test.
    
    Params:
      songs..........The songs DataFrame.
      ratings_train...The subset of ratings used for making predictions. These are the "historical" data.
      ratings_test....The subset of ratings that need to predicted. These are the "future" data.
    Returns:
      A numpy array containing one predicted rating for each element of ratings_test.
    """
    result = []
    for index,row in ratings_test.iterrows():
        mlist = list(ratings_train.loc[ratings_train['userId'] == row['userId']]['songId'])
        csrlist = list(songs.loc[songs['songId'].isin(mlist)]['features'])
        mrlist = list(ratings_train.loc[ratings_train['userId'] ==row['userId']]['rating'])
        cmlist = [cosine_sim(c,songs.loc[songs['songId'] ==row['songId']]['features'].values[0]) for c in csrlist]
        wan = sum([ v*mrlist[i] for i,v in enumerate(cmlist) if v>0 ])
        wadlist = [i for i in cmlist if i>0]
        if (len(wadlist)>0):
            result.append(wan/sum(wadlist))
        else:
            result.append(np.mean(mrlist))
    return np.array(result)

In [294]:
def mean_absolute_error(predictions, ratings_test):
    """
    Return the mean absolute error of the predictions.
    """
    return np.abs(predictions - np.array(ratings_test.rating)).mean()

In [296]:
def main():
    path = ''
    ratings = pd.read_csv(path + os.path.sep + 'ratings.csv')
    songs = pd.read_csv(path + os.path.sep + 'songs.csv')
    songs = tokenize(songs)
    songs, vocab = featurize(songs)
    print('Favorites:')
    print(sorted(vocab.items())[:100])
    ratings_train, ratings_test = train_test_split(ratings)
    print('%d training ratings; %d testing ratings' % (len(ratings_train), len(ratings_test)))
    predictions = make_predictions(songs, ratings_train, ratings_test)
    print('error=%f' % mean_absolute_error(predictions, ratings_test))
    print(predictions[:100])


if __name__ == '__main__':
    main()

Favorites:
[('Agonoize', 0), ('Air', 1), ('Amduscia', 2), ('And One', 3), ('Anorexia Nervosa', 4), ('Artista sconosciuto', 5), ('Behemoth', 6), ('Bella Morte', 7), ('Carpathian Forest', 8), ('Combichrist', 9), ('Covenant', 10), ('Cradle of Filth', 11), ('DIR EN GREY', 12), ('Das Ich', 13), ('Dawn of Ashes', 14), ('Diary of Dreams', 15), ('Dimmu Borgir', 16), ('Duran Duran', 17), ('Emperor', 18), ('Feindflug', 19), ('Funker Vogt', 20), ('God Module', 21), ('Gorgoroth', 22), ('Gothminister', 23), ('Grendel', 24), ('Hocico', 25), ('Hooverphonic', 26), ('Icon of Coil', 27), ('KMFDM', 28), ('Kylie Minogue', 29), ('Laibach', 30), ('Limbonic Art', 31), ('London After Midnight', 32), ('MALICE MIZER', 33), ('Marilyn Manson', 34), ('Mindless Self Indulgence', 35), ('Moi dix Mois', 36), ('Moonspell', 37), ('Morcheeba', 38), ('NoisufX', 39), ('Psyclon Nine', 40), ('Pyotr Ilyich Tchaikovsky', 41), ('Reaper', 42), ('Sopor Aeternus & The Ensemble of Shadows', 43), ('Tamtrum', 44), ('The Crüxshadows',

/Users/krugerbrent/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in double_scalars
  
